In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv('data_dictionary.csv')
df.head()

,Attributs_id,Tabellenr.,Tabelle,Feldname,Felddatentyp,Länge Zeichen,PK,FK,AI,NN,Beschreibung
0,1,1.0,Groesseeneinheit,Groesseeneinheit_id,INTEGER,NaN,X,NaN,X,X,Identifikationsnummer von der Größe
1,2,NaN,Groesseeneinheit,Groesseeneinheit,VARCHAR,2.0,NaN,NaN,NaN,X,Einheit von der Größe
2,3,2.0,Gewichtseinheit,Gewichtseinheit_id,INTEGER,NaN,X,NaN,X,X,Identifikationsnummer vom Gewicht
3,4,NaN,Gewichtseinheit,Gewichtseinheit,VARCHAR,2.0,NaN,NaN,NaN,X,Einheit vom Gewicht
4,5,3.0,Futter_Lagermengen_Einheit,Futter_Lagermengen_Einheit_id,INTEGER,NaN,X,NaN,X,X,Identifikationsnummer vom Futtermengen


In [3]:
df['Länge Zeichen']= df['Länge Zeichen'].fillna(555).astype(int)

In [4]:
df['Feldtyp'] = df.apply(lambda x: f"{x['Felddatentyp']}({x['Länge Zeichen']})"
                         if x['Felddatentyp'] == 'VARCHAR' and x['Länge Zeichen']
                         else x['Felddatentyp'], axis=1)

In [5]:
df2=df[['Tabelle','Feldname','Feldtyp','PK','FK','AI','NN']]
df2.head()

,Tabelle,Feldname,Feldtyp,PK,FK,AI,NN
0,Groesseeneinheit,Groesseeneinheit_id,INTEGER,X,NaN,X,X
1,Groesseeneinheit,Groesseeneinheit,VARCHAR(2),NaN,NaN,NaN,X
2,Gewichtseinheit,Gewichtseinheit_id,INTEGER,X,NaN,X,X
3,Gewichtseinheit,Gewichtseinheit,VARCHAR(2),NaN,NaN,NaN,X
4,Futter_Lagermengen_Einheit,Futter_Lagermengen_Einheit_id,INTEGER,X,NaN,X,X


In [6]:
tabelle = df['Tabelle'].unique()

In [7]:
try:
    conn = sqlite3.connect('projektarbeit.db')
    cursor = conn.cursor()

    cursor.execute("PRAGMA foreign_keys = ON;")

    for tab in tabelle:
       
        tabelle_df=df2[df2['Tabelle']==tab]
    
        columns= []
        foreign_keys = []
    
        for i, row in tabelle_df.iterrows():
            column_def = f"{row['Feldname']} {row['Feldtyp']}"
            if row['NN'] == 'X':
                column_def += ' NOT NULL'
            if row['PK'] == 'X':
                column_def += ' PRIMARY KEY AUTOINCREMENT'

            if row['FK'] == 'X':
                ref_row = df2[(df2['Feldname'] == row['Feldname']) & (df2['PK'] =='X')]
                if not ref_row.empty:
                    ref_table = ref_row.iloc[0]['Tabelle']
                    ref_column = ref_row.iloc[0]['Feldname']
                    #print(ref_table,ref_column)
                    foreign_keys.append(f"FOREIGN KEY ({row['Feldname']}) REFERENCES {ref_table}({ref_column})")  
    
            columns.append(column_def)
        all_columns = columns + foreign_keys
        drop_table_sql = f"DROP TABLE IF EXISTS {tab};"
        create_table_sql = f"CREATE TABLE IF NOT EXISTS {tab}({', '.join(all_columns)});"
        cursor.execute(create_table_sql)
        print(f"Tabelle erstellt: {tab}")
except sqlite3.OperationalError as e:
    print(f"OperationalError: {e}")
finally:
    conn.close()
    

Tabelle erstellt: Groesseeneinheit
Tabelle erstellt: Gewichtseinheit
Tabelle erstellt: Futter_Lagermengen_Einheit
Tabelle erstellt: Gehegegroesse
Tabelle erstellt: Waehrung
Tabelle erstellt: Anrede
Tabelle erstellt: Titel
Tabelle erstellt: Vertretungsgrund
Tabelle erstellt: MwSt
Tabelle erstellt: Im_Zoo_Geboren
Tabelle erstellt: Mitarbeitergeschlecht
Tabelle erstellt: Tiergeschlecht
Tabelle erstellt: Land
Tabelle erstellt: PLZ_Ort
Tabelle erstellt: Futterlieferant
Tabelle erstellt: Futterverbrauch_Kochbuch
Tabelle erstellt: Futterlieferdienst
Tabelle erstellt: Futterauftragsposition
Tabelle erstellt: Lieferadresse
Tabelle erstellt: Futter_Futterliferant
Tabelle erstellt: Futter
Tabelle erstellt: Rundweg
Tabelle erstellt: Mahlzeit
Tabelle erstellt: Mahlzeit_Tier
Tabelle erstellt: Gattung
Tabelle erstellt: Tierart
Tabelle erstellt: Tier
Tabelle erstellt: Tierhistorie
Tabelle erstellt: Tier_Unvertraeglichkeit
Tabelle erstellt: Tier_Krankheit
Tabelle erstellt: Tierkrankheit
Tabelle erstell